In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import theano
from scipy import stats
from sklearn import preprocessing

import plotly.graph_objects as go

In [ ]:
foxes = pd.read_csv('../../data/foxes.csv', sep=';', header=0)
foxes.head()

# 1

> Use a model to infer the total causal influence of `area` on `weight`. Would increasing the area available to each fox make it heavier (healthier)? You might want to standardize the variables. Regardless, use prior predictive simulation to show that your model’s prior predictions stay within the possible outcome range.

Like the prompt suggests, we start by standardizing the appropriate variables.

In [ ]:
features = ['avgfood', 'groupsize', 'area', 'weight']
foxes[features] = preprocessing.scale(foxes[features])

Now, we think about setting our priors in a way that makes sense.

In [ ]:
with pm.Model() as model1:
    
    # data
    weight = foxes['weight']
    group = foxes['group']
    avgfood = foxes['avgfood']
    groupsize = foxes['groupsize']
    area = foxes['area']
    
    # priors
    alpha = pm.Normal('alpha', 0, 10)
    b_area = pm.Normal('b_area', 0, 10)
    b_avgfood = pm.Normal('b_avgfood', 0, 10)
    sigma = pm.Exponential('sigma', 1)
    
    # model
    mu = alpha + b_area * area + b_avgfood * avgfood
    weight_hat = pm.Normal('weight_hat', mu = mu, sigma = sigma, observed = weight)
    
    # sampling stuff
    prior = pm.sample_prior_predictive() # a bunch of draws from the prior for each param
    posterior1 = pm.sample(draws = 1000, tune = 1000)
    posterior_predictive1 = pm.sample_posterior_predictive(posterior1)

In [ ]:
prior_predictive1 = prior['weight_hat']

In [ ]:
fig = go.Figure()

for draw in range(prior_predictive1.shape[0]):
    fig.add_trace(
        go.Scatter(
            x = foxes['area'],
            y = prior_predictive1[draw, :],
            mode = 'lines'
        )
    )
    
fig.update_layout(showlegend = False)
fig.show()